In [1]:
import sys
import os

sys.path.append(os.getcwd())
import numpy as np
import os
import io
import cv2

import torch

from demo.config import (Config,
                    eval_dict_leaf)

from demo.utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

from iv2_utils.iv2 import *

/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvl_clip_vision.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2_clip_vision.py:136: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


No module named 'deepspeed'
deepspeed is not installed!!!


In [2]:
config = Config.from_file('demo/internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [3]:
model_pth = 'model_l14/iv2-stage2.pt'
config['pretrained_path'] = model_pth
config.model.vision_encoder.pretrained = model_pth
config.device = 'cpu'

In [4]:
intern_model, tokenizer = setup_internvideo2(config)

/opt/miniconda3/envs/xclip/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2.py:695: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be exec

load_state_dict: _IncompatibleKeys(missing_keys=[], unexpected_keys=['temp', 'itm_head.weight', 'itm_head.bias'])


In [5]:
def get_logit(video_path, phrase):
    video = cv2.VideoCapture(video_path)
    frames = [x for x in _frame_from_video(video)]
    
    text_candidates = [phrase]
    texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=1, config=config, device = 'cpu')
    return probs[0].item()

In [13]:
get_logit('demo/example1.mp4',"A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.")

32.38642501831055

In [12]:
video = cv2.VideoCapture('demo/example1.mp4')
frames = [x for x in _frame_from_video(video)]

text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]
import time
with torch.no_grad():
    t1 = time.perf_counter()
    texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config, device = 'cpu', log = True)
    t2 = time.perf_counter() - t1

print("Time taken:",t2)
for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

Using Cached
Time taken: 16.858432666
text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 32.3864
text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 29.5699
text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 29.2529
text: A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees. ~ prob: 29.1189
text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 28.9397


In [7]:
from demo.utils import tensor_cache

In [8]:
keys = list(tensor_cache.keys())
print(keys)

['A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.', 'A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.', 'A person dressed in a blue jacket shovels the snow-covered pavement outside their house.', 'A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.', 'A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.', 'A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.', 'A playful dog slides down a snowy hill, wagging its tail with delight.', 'A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.', 'A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.', 'A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery.']


In [16]:
os.listdir('../../..')

['Moo.ipynb',
 'WordFinder',
 'Object.ipynb',
 '.DS_Store',
 'Linear Algebra',
 'Autocomplete',
 'ViCLIP',
 'face',
 'EdgeVid.ipynb',
 'Video Player',
 'tc-clip',
 'InternVideo',
 'photography-model',
 'fine-tuning-2',
 'unmasked_teacher',
 'test.pkl',
 'CoinScraper',
 'leetcode.py',
 'TemporalLanguageRelease',
 'CroppedImages',
 '.ipynb_checkpoints',
 '.git',
 'X-Clip',
 'iv2_utils',
 'video.py',
 'ctcLink-Crawler']

In [18]:
from tqdm.notebook import tqdm

output_dir = 'output'
logits = []
preds  = []
for video_path, phrase, frames in tqdm(pickle_read('../../../photography-model/rustyjar/ACT75.pkl')):
    split_video_to_mp4('../../../photography-model/' + video_path, output_dir=output_dir, window_size=8)

    logit_curr = []
    pbar = tqdm(get_output_dir(output_dir))

    frames_list = {}
    for clip in get_output_dir(output_dir):
        video = cv2.VideoCapture(clip)
        frames_list[clip] = [x for x in _frame_from_video(video)]
    for j in pbar:
        logit = get_logit(j, phrase)
        logit_curr.append(logit)
        if len(logit_curr) > 0:
            pbar.set_description(str(np.argmax(logit_curr) + 1))
    preds.append(np.argmax(logit_curr) + 1)
    logits.append(list(zip(logit_curr, range(1, len(logit_curr) + 1))))

print(preds[:5])
pickle_write(preds, 'InternVideo2-t8.pkl')
pickle_write(logits, 'InternVideo2-logits-t.pkl')

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]


KeyboardInterrupt



In [21]:
import torch

total_params = sum(p.numel() for p in intern_model.parameters() if p.requires_grad)

print(f'Total number of trainable parameters | {total_params}')
print(f'Total parameters                     | {np.sum([int(np.prod(p.shape)) for p in intern_model.parameters()])}')

Total number of trainable parameters | 918528
Total parameters                     | 1410144442
